In [26]:
import sys
#sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *


In [27]:
def get_resids(data, var):
    import statsmodels.formula.api as smf
    model = smf.ols(var + '~ interview_age + C(sex)', data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var):
    data[var + "_r"] = get_resids(data, var)
    return data

In [37]:
bpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/behavioral/'

In [38]:
upps = (readtxt('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/abcd_upps01.txt')
        .rename(columns={'subjectkey': 'ID'})
        .drop(['collection_id','abcd_upps01_id','dataset_id',
              'src_subject_id', 'collection_title', 'interview_date'], axis=1))
#adversitycols = list(full_demos.filter(regex='_exp').columns)
upps_baseline = upps[upps ['eventname'] == 'baseline_year_1_arm_1'].drop('eventname', axis=1).dropna()
upps_y2 = upps[upps['eventname'] == '2_year_follow_up_y_arm_1'].drop('eventname', axis=1).dropna()

upps_baseline['ID'] = upps_baseline['ID'].str.replace('_', '')
upps_y2['ID'] = upps_y2['ID'].str.replace('_', '')

upps_baseline.to_csv(bpath+'upps_baseline.csv', index=False)

In [39]:
upps_baseline_cols = upps_baseline.iloc[:, 3:].columns.to_list()

In [40]:
upps_baseline[['interview_age']+upps_baseline_cols] = upps_baseline[['interview_age']+upps_baseline_cols].apply(pd.to_numeric, errors='coerce')

In [42]:
for i in upps_baseline_cols: upps_baseline = add_resid(upps_baseline, i)
upps_sub_scales = [upps_baseline_cols + "_r" for upps_baseline_cols in upps_baseline_cols]
upps_baseline = upps_baseline[['ID'] + upps_sub_scales]
upps_baseline.to_csv(bpath + 'upps_baseline_r.csv', index=False)